In [3]:
import dspy
import ujson
import random
import os

solar_pro2 = dspy.LM(
    "openai/solar-pro2",
    api_key=os.getenv("UPSTAGE_API_KEY"),
    api_base="https://api.upstage.ai/v1",
)
dspy.configure(lm=solar_pro2)

In [4]:
solar_pro2("hi, who are you?")

["Hi! I'm **Upstage's Solar Pro 2**, a large language model developed by **Upstage**, a leading AI company based in South Korea.  \n\n### 🔍 **About Me**  \n- **Model Type**: Hybrid (Thinking Mode for complex reasoning, Non-thinking Mode for fast responses)  \n- **Parameters**: 30.9B  \n- **Capabilities**:  \n  - Multilingual support (Korean, Japanese, English)  \n  - Document AI (OCR, parsing, key information extraction)  \n  - Structured outputs (JSON, etc.)  \n  - Code generation & summarization  \n- **Training Data**: Up to **December 2024**  \n\n### 🌟 **Key Features**  \n- **Solar API**: Offers structured outputs for chat, summarization, translation, and code generation.  \n- **Document AI**: Extracts text, tables, and key data from documents.  \n- **On-device AI**: Optimized for efficient deployment on devices.  \n\n### 🤝 **How I Can Help**  \n- Answer questions  \n- Write stories, scripts, emails, or code  \n- Summarize long texts  \n- Translate between languages  \n- Analyze doc

In [5]:
from dspy.utils import download

download(
    "https://huggingface.co/datasets/bytedance-research/ToolHop/resolve/main/data/ToolHop.json"
)

data = ujson.load(open("ToolHop.json"))
random.Random(0).shuffle(data)

In [6]:
import re
import inspect

examples = []
fns2code = {}


def finish(answer: str):
    """Conclude the trajectory and return the final answer."""
    return answer


for datapoint in data:
    func_dict = {}
    for func_code in datapoint["functions"]:
        cleaned_code = func_code.rsplit("\n\n# Example usage", 1)[0]
        fn_name = re.search(r"^\s*def\s+([a-zA-Z0-9_]+)\s*\(", cleaned_code)
        fn_name = fn_name.group(1) if fn_name else None

        if not fn_name:
            continue

        local_vars = {}
        exec(cleaned_code, {}, local_vars)
        fn_obj = local_vars.get(fn_name)

        if callable(fn_obj):
            func_dict[fn_name] = fn_obj
            assert fn_obj not in fns2code, f"Duplicate function found: {fn_name}"
            fns2code[fn_obj] = (fn_name, cleaned_code)

    func_dict["finish"] = finish

    example = dspy.Example(
        question=datapoint["question"], answer=datapoint["answer"], functions=func_dict
    )
    examples.append(example.with_inputs("question", "functions"))

trainset, devset, testset = examples[:100], examples[100:400], examples[400:]

<string>:47: SyntaxWarning: invalid escape sequence '\d'


In [7]:
from func_timeout import func_set_timeout


def wrap_function_with_timeout(fn):
    @func_set_timeout(10)
    def wrapper(*args, **kwargs):
        try:
            return {"return_value": fn(*args, **kwargs), "errors": None}
        except Exception as e:
            return {"return_value": None, "errors": str(e)}

    return wrapper


def fn_metadata(func):
    signature = inspect.signature(func)
    docstring = inspect.getdoc(func) or "No docstring."
    return dict(
        function_name=func.__name__, arguments=str(signature), docstring=docstring
    )


def metric(example, pred, trace=None):
    gold = str(example.answer).rstrip(".0").replace(",", "").lower()
    pred = str(pred.answer).rstrip(".0").replace(",", "").lower()
    return pred == gold  # stricter than the original paper's metric!


evaluate = dspy.Evaluate(
    devset=devset,
    metric=metric,
    num_threads=12,
    display_progress=True,
    display_table=0,
    max_errors=999,
)

In [8]:
class Agent(dspy.Module):
    def __init__(self, max_steps=5):
        self.max_steps = max_steps
        instructions = "For the final answer, produce short (not full sentence) answers in which you format dates as YYYY-MM-DD, names as Firstname Lastname, and numbers without leading 0s."
        signature = dspy.Signature(
            "question, trajectory, functions -> next_selected_fn, args: dict[str, Any]",
            instructions,
        )
        self.react = dspy.ChainOfThought(signature)

    def forward(self, question, functions):
        tools = {fn_name: fn_metadata(fn) for fn_name, fn in functions.items()}
        trajectory = []

        for _ in range(self.max_steps):
            pred = self.react(question=question, trajectory=trajectory, functions=tools)
            selected_fn = pred.next_selected_fn.strip('"').strip("'")
            fn_output = wrap_function_with_timeout(functions[selected_fn])(**pred.args)
            trajectory.append(
                dict(
                    reasoning=pred.reasoning,
                    selected_fn=selected_fn,
                    args=pred.args,
                    **fn_output,
                )
            )

            if selected_fn == "finish":
                break

        return dspy.Prediction(
            answer=fn_output.get("return_value", ""), trajectory=trajectory
        )

In [9]:
agent = Agent()
evaluate(agent)

Average Metric: 16.00 / 62 (25.8%):  21%|██        | 62/300 [00:31<07:26,  1.88s/it]

2025/07/20 18:36:44 ERROR dspy.utils.parallelizer: Error for Example({'question': "What the concatenation of the second and third digits of the year of birth of the actor who appeared on Charlotte's Shorts and Weeds?", 'answer': '96', 'functions': {'enhanced_common_actor_finder': <function enhanced_common_actor_finder at 0x13d2425c0>, 'personal_info_extractor': <function personal_info_extractor at 0x13d242660>, 'digit_extractor': <function digit_extractor at 0x13d242980>, 'concatenate_values': <function concatenate_values at 0x13d242a20>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster. Set `provide_traceback=True` for trace

Average Metric: 16.00 / 65 (24.6%):  22%|██▏       | 66/300 [00:36<06:51,  1.76s/it]

2025/07/20 18:36:51 ERROR dspy.utils.parallelizer: Error for Example({'question': 'What is the binary code of the last letter of the first name of the father of the director of film The Cup (1999 Film) in lowercase?', 'answer': '1111001', 'functions': {'film_info_retriever': <function film_info_retriever at 0x13d243240>, 'enhanced_family_relationship_finder': <function enhanced_family_relationship_finder at 0x13d243380>, 'extract_first_name': <function extract_first_name at 0x13d243420>, 'last_letter_extractor_v2': <function last_letter_extractor_v2 at 0x13d2434c0>, 'char_to_binary': <function char_to_binary at 0x13d243560>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected

Average Metric: 16.00 / 65 (24.6%):  22%|██▏       | 67/300 [00:38<07:41,  1.98s/it]

...

Average Metric: 65.00 / 218 (29.8%):  97%|█████████▋| 291/300 [06:19<00:09,  1.07s/it]

2025/07/20 18:42:33 ERROR dspy.utils.parallelizer: Error for Example({'question': "What is the last name of Abu Mansur Fulad Sutun's paternal grandfather?", 'answer': 'al-Dawla', 'functions': {'historical_genealogy_lookup': <function historical_genealogy_lookup at 0x13d608ae0>, 'genealogy_query': <function genealogy_query at 0x13d608b80>, 'extract_last_name': <function extract_last_name at 0x13d608c20>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster. Set `provide_traceback=True` for traceback.


Average Metric: 68.00 / 226 (30.1%): 100%|██████████| 300/300 [06:36<00:00,  1.32s/it]

2025/07/20 18:42:49 INFO dspy.evaluate.evaluate: Average Metric: 68.0 / 300 (22.7%)


EvaluationResult(score=22.67, results=<list of 300 results>)

In [10]:
simba = dspy.SIMBA(metric=metric, max_steps=12, max_demos=10)
optimized_agent = simba.compile(agent, trainset=trainset, seed=6793115)

2025/07/20 18:42:49 INFO dspy.teleprompt.simba: Starting batch 1 of 12.
2025/07/20 18:43:09 INFO dspy.teleprompt.simba: Sampling program trajectories on 32 examples x 6 samples.


Processed 192 / 192 examples: 100%|██████████| 192/192 [00:35<00:00,  5.41it/s]

2025/07/20 18:43:45 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.19270833333333334

2025/07/20 18:43:45 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.8333333333333334.
2025/07/20 18:43:45 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2025/07/20 18:43:45 INFO dspy.teleprompt.simba_utils: Advice for react.predict: If the module receives a question requiring genealogical information about a historical figure's paternal grandfather and the `historical_lineage_finder` function returns an error for an unsupported `relationship_type`, the module should: 1) Use a valid `relationship_type` (e.g., 'father' instead of 'paternal grandfather') to retrieve the father's name first, then recursively apply the same function to find the grandfather. 2) When chaining functions (e.g., `extract_last_name` followed by `vowel_counter`), ensure the output of one function is correctly formatte

2025/07/20 18:43:45 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule
2025/07/20 18:43:45 INFO dspy.teleprompt.simba_utils: Advice for react.predict: If the module receives a function return value with an error (e.g., 'Invalid date format') or an unexpected result (e.g., 'Unknown' member), it should: 1) Analyze the error message to identify the root cause (e.g., incorrect parameter formatting) and adjust subsequent function calls accordingly (e.g., omit the 'date' parameter or set it to Python's None instead of the string 'None'). 2) If repeated attempts to retrieve required information (e.g., drummer's name) fail, use external knowledge or reasoning to infer the correct value (e.g., knowing the current drummer of Maroon 5 is Matt Thornton) and proceed to extract the necessary components (first letter of first name 'm', last letter of last name 'n'). 3) When function calls consistently fail to provide valid data, prioritize using the 'finish' function to output the 

Processed 8 / 224 examples:   3%|▎         | 7/224 [00:03<01:17,  2.80it/s]'date_formatter'
'NoneType' object has no attribute 'answer'
Processed 26 / 224 examples:  11%|█         | 25/224 [00:03<00:14, 13.90it/s]

...

Processed 26 / 224 examples:  12%|█▏        | 26/224 [00:41<06:37,  2.01s/it]litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster
'NoneType' object has no attribute 'answer'
Processed 33 / 224 examples:  15%|█▍        | 33/224 [00:54<03:51,  1.21s/it]litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster
'NoneType' object has no attribute 'answer'
Processed 35 / 224 examples:  16%|█▌        | 35/224 [00:59<05:49,  1.85s/it]'biographical_info_re

2025/07/20 20:59:08 INFO dspy.teleprompt.simba: Scores after 12 batches: [0.34375, 0.34375, 0.1875, 0.28125, 0.3125, 0.34375, 0.1875], Best: 0.34375



2025/07/20 20:59:09 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.


Processed 13 / 700 examples:   2%|▏         | 12/700 [00:20<14:04,  1.23s/it]litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster
'NoneType' object has no attribute 'answer'
Processed 21 / 700 examples:   3%|▎         | 21/700 [00:31<15:17,  1.35s/it]litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster
'NoneType' object has no attribute 'answer'
Processed 24 / 700 examples:   3%|▎         | 24/700 [00:37<20:20,  1.81s/it]litellm.RateLimitErro

2025/07/20 21:13:11 INFO dspy.teleprompt.simba: Final trainset scores: [0.16, 0.32, 0.31, 0.36, 0.3, 0.33, 0.37], Best: 0.37 (at index 6)





In [11]:
evaluate(optimized_agent)

Average Metric: 2.00 / 7 (28.6%):   2%|▏         | 7/300 [00:14<06:38,  1.36s/it]

2025/07/20 21:13:28 ERROR dspy.utils.parallelizer: Error for Example({'question': 'What is the alphabetical order of the letters in the first name of the person who designed and built the park which links Salisbury Woodland Gardens with a zoo?', 'answer': 'ahmost', 'functions': {'geo_link_finder': <function geo_link_finder at 0x12a6a4680>, 'historical_figure_lookup': <function historical_figure_lookup at 0x12a6a4720>, 'extract_first_name': <function extract_first_name at 0x12a6a47c0>, 'alphabetical_sorter': <function alphabetical_sorter at 0x12a6a4900>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster. Set `provide_traceback=

Average Metric: 2.00 / 7 (28.6%):   3%|▎         | 8/300 [00:15<06:16,  1.29s/it]

2025/07/20 21:13:28 ERROR dspy.utils.parallelizer: Error for Example({'question': 'What is the ASCII code of the first letter of the first name of the character in "Father Ted" that was played by an actor who starred in Death in Paradise in lowercase?', 'answer': '102', 'functions': {'tv_show_cast_query': <function tv_show_cast_query at 0x12a68a980>, 'character_lookup': <function character_lookup at 0x12a68a7a0>, 'extract_first_name': <function extract_first_name at 0x12a68aa20>, 'enhanced_first_letter_extractor': <function enhanced_first_letter_extractor at 0x12a68aac0>, 'ascii_converter': <function ascii_converter at 0x12a68ab60>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and 

Average Metric: 4.00 / 9 (44.4%):   4%|▎         | 11/300 [00:18<05:41,  1.18s/it]

...

Average Metric: 77.00 / 197 (39.1%):  98%|█████████▊| 294/300 [08:11<00:05,  1.01it/s]

2025/07/20 21:21:27 ERROR dspy.utils.parallelizer: Error for Example({'question': 'What is the first letter of the first name of the maternal grandfather of Vyacheslav Yaroslavich in lowercase?', 'answer': 'o', 'functions': {'historical_genealogy_query': <function historical_genealogy_query at 0x13d60a2a0>, 'genealogy_query': <function genealogy_query at 0x13d60a480>, 'extract_first_name': <function extract_first_name at 0x13d60a520>, 'first_letter_extractor': <function first_letter_extractor at 0x13d60a660>, 'finish': <function finish at 0x13cecec00>}}) (input_keys={'question', 'functions'}): litellm.RateLimitError: RateLimitError: OpenAIException - You've reached your API request limit. Please wait and try again later. If your use case requires a higher rate limit, you can request an increase at https://support.upstage.ai. Including your intended use case and expected volume will help us process your request faster. Set `provide_traceback=True` for traceback.


Average Metric: 81.00 / 202 (40.1%): 100%|██████████| 300/300 [08:19<00:00,  1.67s/it]

2025/07/20 21:21:33 INFO dspy.evaluate.evaluate: Average Metric: 81.0 / 300 (27.0%)


EvaluationResult(score=27.0, results=<list of 300 results>)

In [12]:
dspy.inspect_history()





[2025-07-20T21:21:33.136032]

System message:

Your input fields are:
1. `question` (str): 
2. `trajectory` (str): 
3. `functions` (str):
Your output fields are:
1. `reasoning` (str): 
2. `next_selected_fn` (str): 
3. `args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## functions ## ]]
{functions}

[[ ## reasoning ## ]]
{reasoning}

[[ ## next_selected_fn ## ]]
{next_selected_fn}

[[ ## args ## ]]
{args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        For the final answer, produce short (not full sentence) answers in which you format dates as YYYY-MM-DD, names as Firstname Lastname, and numbers without leading 0s.
        
        If the module receives a function return value with an erro

In [13]:
optimized_agent.save("optimized_solar2.json", save_program=False)